## This notebook describes my imputation procedure for filling in missing values in my edge properties.

In [1]:
# Imports and miscellany

import csv
import itertools
import pickle
import string
import graphHandler as gh
import inventoryCleaner as iC
import networkx as nx
import numpy as np
import pandas as pd
import more_itertools
from matplotlib import pyplot as plt

%matplotlib inline

# Helper functions

def representsInt(s):
    try: 
        int(s)
        return True
    except ValueError:
        return False

def buildRange(x):
    result = []
    if pd.isnull(x)==True:
        return np.nan
    for part in x.split(','):
        if '-' in part:
            a, b = part.split('-')
            if (representsInt(a)==False):
                strarr = []
                for i in range(ord(a), ord(b)+1):
                    strarr.append(chr(i))
                result.extend(strarr)
            else:
                a, b = int(a), int(b)
                result.extend(range(a, b + 1))
        else:
            if (representsInt(part)==False):
                result.append(part)
            else:
                a = int(part)
                result.append(a)
    return result

def existin(x):
    r = reqs[reqs['Variable']==x]['Reasonable range'].tolist()
    return r

def returnProps(net, u, v, prop):
    return net[u][v][prop]

def returnNeighborProps(net,u,v,props):
    propertyTracker = {el:[] for el in props}
    neighbors = net.neighbors(u)
    for n in neighbors:
        for p in props:
            newp = returnProps(net, u, n, p)
            if (type(newp)=='NoneType')==False:
                propertyTracker[p].append(newp)
    neighbors = net.neighbors(v)
    for n in neighbors:
        for p in props:
            newp = returnProps(net, n, v, p)
            if (type(newp)=='NoneType')==False:
                propertyTracker[p].append(newp)
    return propertyTracker

def getConsensus(props, reqs):
# This function takes in a dict of missing properties and returns the appropriate value after
# accounting for categorical or numerical.
    bestGuess = {el:0 for el in props}
    for key in props:
        iscategorical = reqs.ix[reqs['Variable']==key, 'Categorical']
        if iscategorical.values[0]:
            bestGuess[key]=np.random.choice(props[key])
        else:
            bestGuess[key]=np.mean(props[key])
    return bestGuess

In [2]:
# The first step is to bring in a CSV full of properties and requirements for my data set. 

pathtoreqs = '/home/louisf/Documents/Insight/massdriver/docs/road_inventory_params.csv'
reqs = pd.read_csv(pathtoreqs, delimiter = '^')

In [3]:
# Do a little bit of cleaning to convert some stuff and drop some rows
reqs['Reasonable range'] = reqs['Reasonable range'].map(buildRange)
reqs = reqs[reqs['Disqualify']==False]

In [4]:
# Load in my database 
fpath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RoadInventory.geojson'
pdinv = iC.PDInventory('roadInventory', 'test')
pdinv.geojsontodf(fpath)

toy = pdinv.table
for colname in toy.columns:
    if (sum(colname==reqs['Variable'])!=1):
        toy.drop(colname, inplace=True, axis=1)
        


In [5]:
# Check on how many of the remaining features exceed some threshold of in-range values.

df = pd.DataFrame(index=toy.columns,columns=['value', 'hasbounds'])

for colname in toy.columns:
    nrange = existin(colname)[0]
    if type(nrange)!=float:
        df.loc[colname]= sum(toy[colname].isin(nrange))
        
for row in df.index:
    r = existin(row)[0]
    if type(r) == float:
        df.loc[row]['hasbounds'] = False
    else:
        df.loc[row]['hasbounds'] = True

# Examine only entries with bounds for removal:

df.loc[df['hasbounds']==True]
df.loc[df['hasbounds']==True].value.plot(kind='bar')

# Drop all entries with fewer than 10% of values within bounds
df = df[df['value']<50000]

# The indices of df correspond to all of the properties we will drop from our table.

# Based on this inspection, we will drop:
# fromcity
# fromstate
# fromstreet
# iristatus
# jurisdicti
# opposingdi
# oppositenu
# tostate

# This removal occurs in road_inventory_params.

KeyboardInterrupt: 

In [ ]:
# Next, I will load in my graph. I will use the pickle I generated previously.
picklepath = '/home/louisf/Documents/Insight/massdriver/notebooks/graph_with_risk2.pickle'
graph = nx.read_gpickle(picklepath)

In [ ]:
newNet = graph.net

In [6]:
# Just kidding! I need to generate a new graph with all of the properties I am interested in.

graph = gh.NetworkGenerator()
filepath = '/home/louisf/Documents/Insight/massdriver/data/raw/shapefile/RI_converted.shp'
graph.loadGraph(filepath=filepath, fields=toy.columns.tolist(), simplify=True)
#newNet = graph.net

good = 481430, bad = 30


In [7]:
# Iterate over each edge. Find the properties on that edge that are missing. 
importantProperties = toy.columns.tolist()
nrange = reqs['Reasonable range']
it=0
for u,v in graph.net.edges_iter():
    g = graph.net[u][v]
    missingProps = []
    for props in importantProperties:
        nrange = reqs.loc[reqs['Variable']==props]['Reasonable range']
        if len(nrange.tolist())>0:
            if type(nrange.tolist()[0])!=float:
                if ((g[props] in nrange.tolist()[0])==False):
                    missingProps.append(props)
    neighborProps = returnNeighborProps(graph.net, u, v, missingProps)
    newvals = getConsensus(neighborProps, reqs)
    for key, value in newvals.items():
        graph.net[u][v][key]=value
    if(it%10000==0):
        print(it)
    it+=1


0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000


In [ ]:
graph.net[u][v]

In [8]:
dictlist = []

for u,v in graph.net.edges_iter():
    nd = {}
    for var in reqs['Variable']:
        nd[var]=graph.net[u][v][var]
    dictlist.append(nd)
    

In [9]:
newDB = pd.DataFrame(dictlist)

In [10]:
newDB.to_csv('/home/louisf/Documents/Insight/massdriver/data/imputedDF.csv')

In [11]:
nx.write_gpickle(graph.net, '/home/louisf/Documents/Insight/massdriver/data/largeGraph_filled2.pickle')

In [ ]:
nx.write_shp(graph.net, '/home/louisf/Documents/Insight/massdriver/data/')

In [17]:
newNet = nx.read_gpickle('/home/louisf/Documents/Insight/massdriver/data/largeGraph_filled2.pickle')

In [18]:
# This block traverses a graph and removes most of the properties for storage reasons.


to_keep = ['roadsegmen','assignedle','Json', 'Wkb', 'Wkt']
for u,v in newNet.edges_iter():
    for key in newNet.edge[u][v].copy():
        if key not in to_keep:
            del newNet.edge[u][v][key]

In [19]:
nx.write_gpickle(newNet, '/home/louisf/Documents/Insight/massdriver/data/latest_reduced_needs_risk.pickle')

In [ ]:
# For each node on the edge, find the properties of the edges attached to that node.

### This section is to test functionality of returning path weights.

In [2]:
    net = nx.read_gpickle('/home/louisf/Documents/Insight/massdriver/data/filled_reduced_needs_risk.pickle')
# net = nx.read_gpickle('/home/louisf/Documents/Insight/massdriver/data/largeGraph_filled.pickle')
# net = nx.read_gpickle('/home/louisf/Documents/Insight/massdriver/notebooks/graph_with_risk.pickle')

In [ ]:
# Set up a test path
weight = 'assignedle'
path = gh.pathingSolution(net, 42.3598991, -71.05829879999999, 42.296797, -71.2923877, weight=weight)


In [ ]:
# Traverse the graph and sum up the weights
newpath = gh.pathAlign(net, path)

In [ ]:
gh.findClosestNode(net, 42.3598991, -71.058298799999)

In [ ]:
import scipy.spatial as sp

In [ ]:
points = np.asarray(net.nodes())

In [ ]:
tree = gh.generateKDTree(net)

In [ ]:
dist, indexes = tree.query(path)

In [ ]:
type(points[indexes])

In [ ]:
n = (-71.289882048538956, 42.297668996193117)

In [3]:
path =list(map(tuple, [[-71.29238000000001, 42.29677], [-71.29048, 42.2974], [-71.29018, 42.29751], [-71.28997000000001, 42.297610000000006], [-71.28966000000001, 42.2978], [-71.28948000000001, 42.29796], [-71.28931, 42.29816], [-71.28824, 42.29974000000001], [-71.28751000000001, 42.3008], [-71.28699, 42.30147], [-71.2866, 42.30194], [-71.28612000000001, 42.302490000000006], [-71.28562000000001, 42.303020000000004], [-71.28500000000001, 42.303650000000005], [-71.28468000000001, 42.303940000000004], [-71.28429000000001, 42.30422], [-71.28290000000001, 42.305240000000005], [-71.28204000000001, 42.305870000000006], [-71.28058, 42.30686], [-71.27933, 42.307750000000006], [-71.27857, 42.308310000000006], [-71.27821, 42.30861], [-71.27695, 42.30984], [-71.27619, 42.31062], [-71.27600000000001, 42.31078], [-71.27544, 42.31136], [-71.27506000000001, 42.31175], [-71.27421000000001, 42.31264], [-71.27392, 42.31295], [-71.27369, 42.31317000000001], [-71.27339, 42.31347], [-71.27269000000001, 42.31396], [-71.27168, 42.31456], [-71.2703, 42.31548], [-71.26913, 42.316250000000004], [-71.26762000000001, 42.31732], [-71.26621, 42.3183], [-71.26541, 42.3188], [-71.26504, 42.31908000000001], [-71.26487, 42.319230000000005], [-71.26455, 42.31956], [-71.26443, 42.319720000000004], [-71.2643, 42.319900000000004], [-71.26419, 42.32009], [-71.2637, 42.32105000000001], [-71.26361, 42.32126], [-71.2634, 42.32182], [-71.26318, 42.32235], [-71.26291, 42.32285], [-71.26265000000001, 42.32323], [-71.2624, 42.32358000000001], [-71.26214, 42.323840000000004], [-71.26123000000001, 42.324650000000005], [-71.26104000000001, 42.32481000000001], [-71.26066, 42.32499000000001], [-71.26033000000001, 42.32509], [-71.26009, 42.325140000000005], [-71.25907000000001, 42.325230000000005], [-71.25797, 42.325340000000004], [-71.25769000000001, 42.32537000000001], [-71.25783000000001, 42.325630000000004], [-71.25803, 42.32594], [-71.25852, 42.326480000000004], [-71.25892, 42.32688], [-71.25907000000001, 42.32696000000001], [-71.25938000000001, 42.327070000000006], [-71.2613, 42.32772000000001], [-71.26285, 42.328250000000004], [-71.26668000000001, 42.32961], [-71.26699, 42.32977], [-71.26763000000001, 42.33017], [-71.26776000000001, 42.330290000000005], [-71.26788, 42.33044], [-71.26823, 42.33117], [-71.26877, 42.332440000000005], [-71.26884000000001, 42.332660000000004], [-71.26885, 42.33288], [-71.26882, 42.333070000000006], [-71.26862000000001, 42.33381000000001], [-71.26839000000001, 42.33465], [-71.26826000000001, 42.334990000000005], [-71.26787, 42.335840000000005], [-71.26725, 42.337390000000006], [-71.2672, 42.337540000000004], [-71.26719, 42.337680000000006], [-71.26692, 42.33785], [-71.26672, 42.337860000000006], [-71.26634, 42.33789], [-71.26595, 42.33796], [-71.26572, 42.338010000000004], [-71.26549, 42.33811], [-71.26504, 42.33836], [-71.26453000000001, 42.3387], [-71.26395000000001, 42.33905], [-71.26275000000001, 42.339830000000006], [-71.26135000000001, 42.34076], [-71.26075, 42.34114], [-71.26033000000001, 42.341370000000005], [-71.26004, 42.3415], [-71.25983000000001, 42.341590000000004], [-71.25921000000001, 42.341820000000006], [-71.25864, 42.34199], [-71.25638000000001, 42.34257], [-71.25557, 42.34279], [-71.25512, 42.34295], [-71.25423, 42.34337], [-71.25364, 42.34371], [-71.25321000000001, 42.344], [-71.25215, 42.34469], [-71.25181, 42.3449], [-71.25122, 42.34519], [-71.25068, 42.34541], [-71.25019, 42.34557], [-71.24971000000001, 42.3457], [-71.24899, 42.345850000000006], [-71.2484, 42.34593], [-71.24197000000001, 42.34653], [-71.23795000000001, 42.346880000000006], [-71.23530000000001, 42.346970000000006], [-71.23421, 42.34707], [-71.23209, 42.347390000000004], [-71.23013, 42.34761], [-71.22837000000001, 42.347770000000004], [-71.22579, 42.34801], [-71.22452000000001, 42.348130000000005], [-71.22352000000001, 42.348240000000004], [-71.2226, 42.348380000000006], [-71.22196000000001, 42.3485], [-71.22062000000001, 42.34879], [-71.21675, 42.34966], [-71.21538000000001, 42.34993], [-71.21442, 42.35007], [-71.2112, 42.350500000000004], [-71.20956000000001, 42.35071000000001], [-71.20713, 42.351090000000006], [-71.20606000000001, 42.3513], [-71.20450000000001, 42.35164], [-71.20333000000001, 42.35192000000001], [-71.20225, 42.35222], [-71.19566, 42.35414], [-71.19406000000001, 42.354560000000006], [-71.19308000000001, 42.35477], [-71.19030000000001, 42.35537], [-71.18875, 42.35569], [-71.18801, 42.35584], [-71.1855, 42.35627], [-71.18365, 42.35665], [-71.18201, 42.357020000000006], [-71.18056, 42.357330000000005], [-71.1798, 42.357490000000006], [-71.17909, 42.357620000000004], [-71.17814, 42.35772], [-71.17693000000001, 42.357760000000006], [-71.17634000000001, 42.35775], [-71.17335, 42.35761], [-71.16859000000001, 42.357420000000005], [-71.16684000000001, 42.35739], [-71.16486, 42.357420000000005], [-71.16226, 42.3575], [-71.16204, 42.357510000000005], [-71.15815, 42.35763], [-71.1575, 42.35766], [-71.15462000000001, 42.35775], [-71.15316, 42.35779], [-71.15191, 42.35783000000001], [-71.15091000000001, 42.3579], [-71.14877, 42.35815], [-71.14784, 42.35824], [-71.14755000000001, 42.35826], [-71.14675000000001, 42.358270000000005], [-71.14607000000001, 42.358250000000005], [-71.14542, 42.35819], [-71.13857, 42.35732], [-71.13815000000001, 42.35727000000001], [-71.13506000000001, 42.356860000000005], [-71.13146, 42.35638], [-71.13090000000001, 42.35633000000001], [-71.13061, 42.356320000000004], [-71.12997, 42.356320000000004], [-71.12926, 42.35636], [-71.12871000000001, 42.35643], [-71.12805, 42.356550000000006], [-71.12768000000001, 42.356640000000006], [-71.12624000000001, 42.35699], [-71.12554, 42.35712], [-71.12434, 42.35723], [-71.123, 42.35725], [-71.12237, 42.357220000000005], [-71.12189000000001, 42.35717], [-71.12140000000001, 42.357110000000006], [-71.12079, 42.357000000000006], [-71.12046000000001, 42.35692], [-71.12026, 42.35685], [-71.11999, 42.35672], [-71.11973, 42.35656], [-71.11945, 42.35634], [-71.11924, 42.35611], [-71.11897, 42.35573], [-71.11885000000001, 42.35551], [-71.11882, 42.35537], [-71.11838, 42.3547], [-71.11832000000001, 42.354670000000006], [-71.11819000000001, 42.35448], [-71.11785, 42.354110000000006], [-71.11744, 42.353790000000004], [-71.11699, 42.35351], [-71.11396, 42.35193], [-71.11347, 42.3517], [-71.11295000000001, 42.351510000000005], [-71.11243, 42.3513], [-71.11193, 42.351060000000004], [-71.10995000000001, 42.34996], [-71.10944, 42.34971], [-71.10893, 42.349500000000006], [-71.1084, 42.34931], [-71.10785, 42.349140000000006], [-71.10729, 42.349000000000004], [-71.10673, 42.348890000000004], [-71.10157000000001, 42.3479], [-71.10088, 42.34781], [-71.10025, 42.347750000000005], [-71.09966, 42.347730000000006], [-71.09035, 42.34778], [-71.08981, 42.34776], [-71.08936, 42.347730000000006], [-71.08889, 42.34767], [-71.08801000000001, 42.347570000000005], [-71.08761000000001, 42.347550000000005], [-71.08138000000001, 42.347640000000006], [-71.07932000000001, 42.34767], [-71.07748000000001, 42.34772], [-71.07369, 42.34783], [-71.06978000000001, 42.34783], [-71.06948000000001, 42.34783], [-71.06918, 42.347820000000006], [-71.06859, 42.34778], [-71.06657000000001, 42.34752], [-71.06598000000001, 42.34743], [-71.06544000000001, 42.34731], [-71.06419000000001, 42.347], [-71.06391, 42.346940000000004], [-71.06245000000001, 42.34662], [-71.06206, 42.346560000000004], [-71.06174, 42.34651], [-71.06146000000001, 42.346470000000004], [-71.06093, 42.346410000000006], [-71.06038000000001, 42.34637], [-71.05992, 42.346360000000004], [-71.05951, 42.34639000000001], [-71.05914, 42.346450000000004], [-71.05883, 42.34653], [-71.05839, 42.346700000000006], [-71.05814000000001, 42.34684], [-71.05794, 42.347], [-71.05777, 42.34716], [-71.05756000000001, 42.3474], [-71.05744, 42.34763], [-71.05716000000001, 42.3485], [-71.05697, 42.349120000000006], [-71.05683, 42.34942], [-71.05619, 42.350680000000004], [-71.05576, 42.35161], [-71.05566, 42.35197], [-71.05547, 42.35226], [-71.05529, 42.35249], [-71.05507, 42.352700000000006], [-71.05481, 42.352900000000005], [-71.05446, 42.353100000000005], [-71.05404, 42.353300000000004], [-71.05174000000001, 42.354350000000004], [-71.05141, 42.35452], [-71.05113, 42.35472], [-71.05092, 42.354890000000005], [-71.05075000000001, 42.35512000000001], [-71.05047, 42.35564], [-71.05039000000001, 42.355940000000004], [-71.05039000000001, 42.356280000000005], [-71.05047, 42.35665], [-71.05058000000001, 42.357000000000006], [-71.05074, 42.3573], [-71.05102000000001, 42.357780000000005], [-71.05153, 42.35850000000001], [-71.05173, 42.35891], [-71.05168, 42.359030000000004], [-71.05183000000001, 42.359440000000006], [-71.05212, 42.36007], [-71.05223000000001, 42.360240000000005], [-71.05236000000001, 42.36037], [-71.05245000000001, 42.36047000000001], [-71.05265, 42.36065000000001], [-71.05271, 42.36068], [-71.0528, 42.36075], [-71.05296, 42.360870000000006], [-71.05328, 42.3611], [-71.05424000000001, 42.36159000000001], [-71.05449, 42.361700000000006], [-71.05465000000001, 42.361740000000005], [-71.05488000000001, 42.36173], [-71.05499, 42.36169], [-71.05510000000001, 42.36162], [-71.05518000000001, 42.36157], [-71.05539, 42.36133], [-71.05554000000001, 42.36112000000001], [-71.05589, 42.360730000000004], [-71.05607, 42.36061], [-71.05644000000001, 42.36045], [-71.05716000000001, 42.36027000000001], [-71.05730000000001, 42.360240000000005], [-71.05726, 42.36009000000001], [-71.05703000000001, 42.359280000000005], [-71.05689000000001, 42.358880000000006], [-71.05721000000001, 42.358850000000004], [-71.05771, 42.358830000000005], [-71.05782, 42.358830000000005], [-71.05824000000001, 42.358920000000005], [-71.05861, 42.359], [-71.05892, 42.35911], [-71.05930000000001, 42.35922], [-71.0595, 42.35933], [-71.05963000000001, 42.35942], [-71.05968, 42.35952], [-71.05981, 42.359820000000006]]))


In [4]:
p2 = gh.pathAlign(net, path)

starting the path alignment
ending path alignment, 0.8157689571380615


In [5]:
p2

[(-71.292145228653681, 42.296852234559893),
 (-71.289882048538956, 42.297668996193117),
 (-71.289610033471888, 42.297869970440161),
 (-71.287942494558095, 42.300181228294903),
 (-71.287334436154822, 42.301034078431663),
 (-71.286590655640708, 42.301947182093805),
 (-71.285534423669716, 42.303095787199872),
 (-71.285206079657655, 42.303427274204722),
 (-71.284730005276558, 42.30486000066206),
 (-71.283439999981752, 42.306000002819552),
 (-71.281630003944642, 42.306839999393205),
 (-71.280589950126341, 42.30686000551178),
 (-71.279790002207136, 42.308430003259431),
 (-71.278588024381321, 42.308307962078274),
 (-71.27822000229196, 42.308620029439403),
 (-71.276758633187171, 42.310023440183755),
 (-71.276150003528358, 42.310640001634688),
 (-71.276019982175214, 42.310760009875587),
 (-71.275077589986154, 42.311740783292102),
 (-71.274212429338121, 42.31263916771784),
 (-71.27377132171631, 42.313092791117505),
 (-71.272691363874443, 42.313978004869092),
 (-71.272017848301957, 42.31440991180

In [27]:
newp = linkShortPaths(net, p2, "assignedle")

({(-71.292145228653681, 42.296852234559893): 0, (-71.29184100186349, 42.29696000242845): 0.0179, (-71.29429350722947, 42.29641503630313): 0.11660000000000001, (-71.29253522699315, 42.2958947763098): 0.1003, (-71.29357039150955, 42.29736951923707): 0.0902, (-71.29323998233087, 42.29564002583206): 0.1152, (-71.29422945635872, 42.2964206834823): 0.0947, (-71.2909600000473, 42.2954400008678): 0.12399999999999999, (-71.29150575547834, 42.29638089758609): 0.057999999999999996, (-71.29352000101026, 42.29711000141736): 0.07150000000000001, (-71.29117608730654, 42.29580284357237): 0.09789999999999999, (-71.28988204853896, 42.29766899619312): 0.13369999999999999, (-71.29300377893937, 42.29658421636784): 0.0461}, {(-71.292145228653681, 42.296852234559893): [(-71.292145228653681, 42.296852234559893)], (-71.29184100186349, 42.29696000242845): [(-71.292145228653681, 42.296852234559893), (-71.29184100186349, 42.29696000242845)], (-71.29429350722947, 42.29641503630313): [(-71.292145228653681, 42.29685

KeyboardInterrupt: 

In [34]:
def linkShortPaths(network, path, weight):
    newpath = []
    it = 0
    for node in path:
        it += 1
        if it == len(path):
            return newpath
        if net.has_edge(node,path[it])==False:
            try:
               # np = gh.getShortestPath(network, node, path[it], weight=weight)
                dist, np = nx.single_source_dijkstra(network, node, target=path[it], cutoff=30, weight=weight)
                #print(np)
                newpath.append(np)
            except nx.NetworkXNoPath as netx:
                print(netx.args)
        else:
            newpath.append(node)


In [29]:
np

[(-71.287942494558095, 42.300181228294903),
 (-71.28785259794424, 42.30031054314406),
 (-71.28733443615482, 42.30103407843166)]

In [35]:
#path = gh.pathAlign(net, path)

n2 = linkShortPaths(net, p2, 'assignedle')

In [38]:
n2[0]

{(-71.2950355972711,
  42.2962977877746): [(-71.292145228653681,
   42.296852234559893), (-71.29300377893937, 42.29658421636784), (-71.29422945635872,
   42.2964206834823), (-71.29429350722947,
   42.29641503630313), (-71.2950355972711, 42.2962977877746)],
 (-71.2949001502393,
  42.2969590969538): [(-71.292145228653681,
   42.296852234559893), (-71.29300377893937,
   42.29658421636784), (-71.29422945635872,
   42.2964206834823), (-71.29429350722947,
   42.29641503630313), (-71.2949001502393, 42.2969590969538)],
 (-71.29481000076673,
  42.29566999973323): [(-71.292145228653681,
   42.296852234559893), (-71.29300377893937,
   42.29658421636784), (-71.29323998233087,
   42.29564002583206), (-71.29481000076673, 42.29566999973323)],
 (-71.29429350722947,
  42.29641503630313): [(-71.292145228653681,
   42.296852234559893), (-71.29300377893937,
   42.29658421636784), (-71.29422945635872,
   42.2964206834823), (-71.29429350722947, 42.29641503630313)],
 (-71.29422945635872,
  42.2964206834823):

In [ ]:
nx.shortest_path(net, (-71.292145228653681, 42.296852234559893),(-71.289882048538956, 42.297668996193117), weight='assignedle')